# **Correlation analysis**
*This script performs correlation analysis between the average of each fitted parameter (within a single structure), for each subject and the total score of the PHQ questionnaire (questionnaire on symptoms of depression)*

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
import csv

### **Loading necassary data**
*The questionnaire scores are saved in an excel file. This file is loaded and the total score of each participant is calculated to be used in correlation analysis*

*The merged file containing fitted parameters from every participant and every structure is loaded*

*Column names of the fitted parameter file is also extracted for later processing*

In [ ]:
df_PHQ = pd.read_excel('/hus/home/hining/data/Patient data template.xlsx', sheet_name='PHQ-9', skiprows=[0,1], 
                       usecols=[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18] )
PHQ_scores = df_PHQ.sum(axis=1).to_numpy()

df = pd.read_json('Fitted_parameters.json', orient='index')
parameter_names = list(df.loc['IFMM_001', 'Thalamus'][0].keys())

### **Analysis**
*The correlation analysis is only performed between the scores and the average parameter value of four selected parameters; the small sphere diameter, the large sphere diameter, the stick dispersion parameter and the volume fraction of the stick compartment* 

*The cell below loops through every subcortical structure and performs the correlation analysis on the PHQ scores and the average parameter value for these parameters. Based on earlier evaluation of the distribution of fitted parameters all parameters close to max/min is excluded before calculating the average. For the diameters this means all values close to max/min of range, and for the stick dispersion and volume fraction all values very close to either 0 or 1*

*The correlation results (both correlation and p-value) is stored to a list of lists along with the name of the subcortical structure and the parameter. When all correlation analysis are done the list of lists is saved as a csv file.*

In [ ]:
correlation_results = [['Subcortical structure', 'Parameter', 'Spearman_r', 'P-value']] 
indices = [0, 1, 5, 6] # idices of desired parameters

for column in df.columns:

    for i, param in enumerate(parameter_names):
        if i not in indices:
            continue
        mean_vals = []

        if i == 0:
            for row in df.index:
                values = df.loc[row, column][0][param]
                values = [x for x in values if round(x,7) not in [1e-6, 1e-5] ] 
                mean = np.mean(values)
                mean_vals.append(mean)

        elif i == 1:
            for row in df.index:
                values = df.loc[row, column][0][param]
                values = [x for x in values if round(x,7) not in [1e-5, 3e-5]] 
                mean = np.mean(values)
                mean_vals.append(mean)

        elif i == 5:
            for row in df.index:
                values = df.loc[row, column][0][param]
                values = [x for x in values if 0.025 <= x <= 0.975] 
                mean = np.mean(values)
                mean_vals.append(mean)   


        elif i == 6:
            for row in df.index:
                stick_fraction_in_bundle = df.loc[row, column][0][param]
                partial_volume_bundle = df.loc[row, column][0]['partial_volume_3']   
                partial_volume_stick =  [b*s for b, s in zip(partial_volume_bundle, stick_fraction_in_bundle)]
                values = [x for x in partial_volume_stick if 0.025 <= x <= 0.975]
                mean = np.mean(values)
                mean_vals.append(mean)

        # eliminate potetial NAN values
        PHQ_scores = np.array(PHQ_scores, dtype=np.float64)
        mean_vals= np.array(mean_vals, dtype=np.float64)
        non_nan_mask = ~np.isnan(PHQ_scores) & ~np.isnan(mean_vals)
        PHQ_non_nan = PHQ_scores[non_nan_mask] 
        mean_val_non_nan = mean_vals[non_nan_mask]
     
        # Correlation analysis     
        correlation, p_value = spearmanr(PHQ_non_nan, mean_val_non_nan)
        correlation_results.append([column, param, correlation, p_value] )

with open('Correlation_results.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(correlation_results)